# Máquina Vetores de Suporte

Em uma classificação binária linear com dados x,y, com y em {-1,1} e os parâmetros theta e b(bias), o objetivo é encontrar o hiperplano que separa os dados em duas classes distintas, o classificador Máquina Vetores de Suporte, do inglês Support Vector Machine (SVM), pode ser definido como 

<a href="http://www.codecogs.com/eqnedit.php?latex=z&space;=&space;\theta^{T}x&space;&plus;&space;b" target="_blank"><img src="http://latex.codecogs.com/gif.latex?z&space;=&space;\theta^{T}x&space;&plus;&space;b" title="z = \theta^{T}x + b" /></a>

<a href="http://www.codecogs.com/eqnedit.php?latex=h_{\theta,&space;b}(x)&space;=&space;g(z)" target="_blank"><img src="http://latex.codecogs.com/gif.latex?h_{\theta,&space;b}(x)&space;=&space;g(z)" title="h_{\theta, b}(x) = g(z)" /></a>

Onde g(z) = 1 se z ≥ 0 ou $g(z) = −1$ caso contrario.

Um conceito importante no SVM é a margem. Ela é a distância entre o hiperplano e o primeiro ponto de cada classe. O objetivo neste caso é encontrar uma fronteira de decisão que maximize a margem, gerando como resultado um conjunto de previsões com um bom ajuste.

<b> Importes

In [1]:
from sklearn import preprocessing, neighbors, svm
from sklearn.model_selection import train_test_split
from os import listdir
from os.path import isfile, join
import numpy as np
import geopandas as gpd
import pandas as pd
import requests
import io

<b> Configurado os caminhos

In [2]:
falsospath = "/home/sansigolo/Documents/git/CAP-240-394/src/SVM/AQM_L8_221_067_2017/FALSOS_221_067-2017/"

queimadaspath = "/home/sansigolo/Documents/git/CAP-240-394/src/SVM/AQM_L8_221_067_2017/QUEIMADAS_221_067-2017/"

<b> Pegando os nomes de arquivo

In [3]:
falsosfilenames = [y for y in listdir(falsospath) for ending in ['dbf', 'shp', 'prj', 'shx'] if y.endswith(ending)] 

queimadasfilenames = [y for y in listdir(queimadaspath) for ending in ['dbf', 'shp', 'prj', 'shx'] if y.endswith(ending)] 

<b> Imprimindo os nomes

In [4]:
print(falsosfilenames)
print("")
print(queimadasfilenames)

['FALSOS_221_067-2017.dbf', 'FALSOS_221_067-2017.prj', 'FALSOS_221_067-2017.shp', 'FALSOS_221_067-2017.shx']

['QUEIMADAS_221_067-2017.dbf', 'QUEIMADAS_221_067-2017.prj', 'QUEIMADAS_221_067-2017.shp', 'QUEIMADAS_221_067-2017.shx']


<b> Pegandando o Shapefile

In [5]:
f_dbf, f_prj, f_shp, f_shx = [falsosfilename for falsosfilename in falsosfilenames]

falsos = gpd.read_file(falsospath+f_shp)

<b> Pegando o Shapefile

In [6]:
q_dbf, q_prj, q_shp, q_shx = [queimadasfilename for queimadasfilename in queimadasfilenames]

queimadas = gpd.read_file(queimadaspath+q_shp)

<b> Imprimindo o Shape

In [7]:
print("\nFalsos Shape: {}".format(falsos.shape)+"\n")

print("Queimadas Shape: {}".format(queimadas.shape))


Falsos Shape: (8427, 25)

Queimadas Shape: (7243, 25)


<b> Criando o Dataframe

In [8]:
df = pd.concat([falsos, queimadas], ignore_index=True)

df.replace('?', -99999, inplace=True)

<b> Imprimindo o Dataframe

In [9]:
print("\nDataframe tail:\n", df.tail())


Dataframe tail:
              id  cod_sat                cena_id                      nome_arq  \
15665  59258025        8  LC82210672017285LGN00  LC82210672017285LGN00.tar.gz   
15666  59258029        8  LC82210672017285LGN00  LC82210672017285LGN00.tar.gz   
15667  59258034        8  LC82210672017285LGN00  LC82210672017285LGN00.tar.gz   
15668  59258037        8  LC82210672017285LGN00  LC82210672017285LGN00.tar.gz   
15669  59258055        8  LC82210672017285LGN00  LC82210672017285LGN00.tar.gz   

       orb_pto     area_ha  perim                     n_arq_ant      ndvi  \
15665  221_067    2.164134    840  LC82210672017269LGN00.tar.gz  0.272034   
15666  221_067    0.721722    360  LC82210672017269LGN00.tar.gz  0.262785   
15667  221_067    2.525723    900  LC82210672017269LGN00.tar.gz  0.208865   
15668  221_067    2.343879    780  LC82210672017269LGN00.tar.gz  0.236250   
15669  221_067  209.233220  18479  LC82210672017269LGN00.tar.gz  0.210700   

           nbrl                 

<b> Removendo as colunas

In [13]:
df.drop(['id','cod_sat','cena_id', 'area_ha', 'dif_dnbrl', 'medianb6', 'medianb7', 'nome_arq', 'orb_pto', 'n_arq_ant', 'medianb1', 'proc_id','data_atual', 'data_anter', 'geometry', 'focos', 'ndvi', 'perim'], 1, inplace=True)

<b> Imprimindo o dataframe

In [14]:
print("\nDataframe tail:\n", df.tail())


Dataframe tail:
            nbrl  dif_ndvi  medianb2  medianb3  medianb4  medianb5  verifica  \
15665  0.004431  0.196688  0.113548  0.099376  0.110541  0.191617         1   
15666 -0.021231  0.137276  0.121013  0.106137  0.122500  0.211184         1   
15667 -0.079713  0.132524  0.113283  0.101523  0.116598  0.182004         1   
15668 -0.082420  0.119911  0.109639  0.094806  0.110277  0.184493         1   
15669 -0.092585  0.131845  0.100929  0.086746  0.095864  0.153738         1   

           ndwi         bai       nbr  
15665 -0.132762   57.358694 -0.028283  
15666 -0.140264   42.803092 -0.014161  
15667 -0.153531   65.960875 -0.063513  
15668 -0.168651   64.085890 -0.090958  
15669 -0.172573  113.584703 -0.081024  


<b> Configurando X e y

In [15]:
X = np.array(df.drop(['verifica'],1))
y = np.array(df['verifica'])

<b> Fazendo o treinamento e o teste

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

<b> Rodando a Máquina Vetores de Suporte

In [17]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

<b>Imprimindo a acurácia

In [18]:
accuracy = clf.score(X_test, y_test)

print('Accuracy: ', accuracy)

Accuracy:  0.7332482450542438
